# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [3]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [4]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [6]:
# %%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points('Lng', 'Lat', geo=True, tiles='OSM', color='City', alpha=0.6, s='Humidity', scale=1)


# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [10]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE
clean_city_data_df = city_data_df[city_data_df['Humidity'] <= 60]

# Drop any rows with null values
# YOUR CODE HERE
clean_city_data_df.dropna()

# Display sample data
# YOUR CODE HERE
clean_city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230
12,12,trairi,-3.2778,-39.2689,31.07,51,34,8.09,BR,1666108038
22,22,guiratinga,-16.3494,-53.7583,38.35,14,22,3.10,BR,1666108243


### Step 3: Create a new DataFrame called `hotel_df`.

In [12]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE
hotel_df['Hotel Name'] = ''

# Display sample data
# YOUR CODE HERE
hotel_df.head()


,City,Country,Lat,Lng,Humidity,Hotel Name
0,faya,SA,18.3851,42.4509,35,
1,farsund,NO,58.0948,6.8047,100,
2,new norfolk,AU,-42.7826,147.0587,58,
3,jamestown,US,42.0970,-79.2353,77,
4,lanzhou,CN,36.0564,103.7922,48,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [17]:
# Set parameters to search for a hotel
radius = 10000
params = {
    'categories': 'accommodation.hotel',
    'filter': '',
    'bias':'',
    'apiKey':geoapify_key,
    'limit':1}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lon = row['Lng']
    lat = row['Lat']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lon},{lat},{radius}'
    params["bias"] = f'proximity:{lon},{lat}'

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
faya - nearest hotel: No hotel found
farsund - nearest hotel: Rederiet Hotell
new norfolk - nearest hotel: Woodbridge on the Derwent
jamestown - nearest hotel: DoubleTree Jamestown
lanzhou - nearest hotel: 西园宾馆
ushuaia - nearest hotel: Apart Hotel Aires del Beagle
albany - nearest hotel: No hotel found
carnarvon - nearest hotel: No hotel found
sisimiut - nearest hotel: Hotel SØMA
saint-pierre - nearest hotel: Tropic Hotel
colorado - nearest hotel: Servilha Park Hotel
nikolskoye - nearest hotel: No hotel found
trairi - nearest hotel: Pousada Villa Aurora
dobryanka - nearest hotel: Уральская Венеция
qaanaaq - nearest hotel: Hotel Qaanaaq
mataura - nearest hotel: No hotel found
khatanga - nearest hotel: Хатанга
tasiilaq - nearest hotel: Angmagssalik
tiksi - nearest hotel: Арктика
puerto ayora - nearest hotel: Hostal La Mirada De Solitario George
avarua - nearest hotel: Paradise Inn
nuevo laredo - nearest hotel: Best Western Plus
guiratinga - nearest hotel: No hotel f

,City,Country,Lat,Lng,Humidity,Hotel Name
0,faya,SA,18.3851,42.4509,35,No hotel found
1,farsund,NO,58.0948,6.8047,100,Rederiet Hotell
2,new norfolk,AU,-42.7826,147.0587,58,Woodbridge on the Derwent
3,jamestown,US,42.0970,-79.2353,77,DoubleTree Jamestown
4,lanzhou,CN,36.0564,103.7922,48,西园宾馆
...,...,...,...,...,...,...
572,sovetskiy,RU,61.3614,63.5842,96,Кировская
573,tatarsk,RU,55.2190,75.9828,89,Постоялый двор
574,canchungo,GW,12.0672,-16.0333,61,Hotel Pereira
575,iquique,CL,-20.2208,-70.1431,68,Hostal Vivar


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [20]:
%%capture --no-display

# Configure the map plot
hotel_plot = hotel_df.hvplot.points('Lng', 'Lat', geo=True, tiles='OSM', color='City', alpha=0.8, hover=True, hover_cols='all', s='Humidity', scale=1)

# Display the map
hotel_plot


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,index,Country,Hotel Name)